# In Class Lab 1

### Due by 5 PM Jan 31st in your github repository 'Labs/Lab1' folder

## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [2]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

In [3]:
def vlsr(r0, mu=6.379, pec=12.24*(u.km/u.s)):
    """computes velocity at the local standard of rest in km/s
            VLSR = 4.74 * mu * r0 - pec
        inputs:
            r0: 'astropy quantity'
                Distance from sun to galactic center (kpc)
            mu: 'float'
                Proper motion of Sgr A* (mas/yr)
                Default is from Reid & Brunthaler 2004
            pec: 'astropy quantity'
                Peculiar motion of the Sun in the v direction (km/s)
                Default is from Schonrich + 2010
        outputs:
            VLSR: 'astropy quantity'
                Velocity of the local standard of rest (km/s)
    """
    return 4.74 * mu * (r0/u.kpc) * (u.km/u.s) - pec


In [4]:
# define distances to Galactic Center from Sun

r0_reid = 8.34*u.kpc    # Distance from Reid+2014 in kpc
r0_grav = 8.178*u.kpc   # Distance from GRAVITY collab Abuter+2019 in kpc
r0_sg = 7.9*u.kpc       # Distance from texbook Sparke & Gallagher

In [5]:
# compute VLSR using r0 from reid
vlsr_reid = vlsr(r0_reid)
print(f'{vlsr_reid:.3f}')

239.932 km / s


In [6]:
# compute VLSR using r0 from GRAVITY
vlsr_grav = vlsr(r0_grav)
print(f'{vlsr_grav:.3f}')

235.034 km / s


In [7]:
# compute VLSR using r0 from GRAVITY
vlsr_sg = vlsr(r0_sg)
print(f'{vlsr_sg:.3f}')

226.628 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [8]:
def T_orbSun(r, v):
    """ this function computes the orbital period of the sun
            T = 2 * pi * R / v
        Inputs:
            r: 'astropy quantity'
                Distance to Galactic Center (kpc)
            v: 'astropy quantity'
                Velocity (of sun) in v direction (km/s)
        Outputs:
            T: 'astropy quantity'
            Orbital period in Gyr
    """
    v_kpcGyr = v.to(u.kpc / u.Gyr)   # converting v from km/s to kpc/Gyr
    T = 2 * np.pi * r / v_kpcGyr
    return T

In [9]:
# velocity of sun = vlsr + peculiar
v_pec = 12.24 * u.km / u.s
v_sun = vlsr_grav + v_pec

In [10]:
# compute orbital period of sun w/ r0 from grav collab
T_grav = T_orbSun(r0_grav, v_sun)
print(T_grav)

0.20318680562272234 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [11]:
# age of universe / orbital period
age = 13.8 * u.Gyr  # age of universe
print(age/T_grav)

67.91779593023313


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius ($R_o$) in units of $M_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 $kpc^3$/$Gyr^2$/$M_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  $M_\odot$) ? 

In [12]:
# gravitational constant in desired units
G = const.G.to(u.kpc**3 / u.Gyr**2 / u.Msun)

In [13]:
# density profile: rho = vlsr^2 / (4*pi*G*R^2)
# mass = integrate rho dV
#      = rho 4*pi*r**2 dr
#      = vlsr^2 / (4*pi*G*R^2) * (4*pi*r**2) dr
#      = (vlsr^2 / G) * r

def mass_iso(r, vlsr):
    """ function computes dark matter mass enclosed within a given distance assuming
        an isothermal sphere model for dark matter
        Inputs:
            r: 'astropy quantity'
                Distance to Galactic Center (kpc)
            vlsr: 'astropy quantity'
                velocity of local standard of rest (km/s)
        Outputs:
            M: 'astropy quantity'
                mass enclosed within r (Msun)
    """
    vlsr_kpcGyr = vlsr.to(u.kpc/u.Gyr)  # converting km/s to kpc/Gyr
    M = (vlsr_kpcGyr**2 / G) * r          # mass for isothermal sphere
    return M

In [14]:
M_isoSun = mass_iso(r0_grav, vlsr_grav)
print(f'{M_isoSun:.2e}')    # scientific notation

1.05e+11 solMass


In [15]:
# compute mass within 260 kpc
M_iso260 = mass_iso(260*u.kpc, vlsr_grav)
print(f'{M_iso260:.2e}')

3.34e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of $M_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [17]:
# Potential for Hernquist profile: Phi = -GM / (r+a)

# Using the potential for a Hernquist profile, eqn for escape speed becomes:
#   v_esc**2 = 2*G*M / (r+a)

# Rearrange escape speed eqn for M:
#   M = v_esc**2 * (r+a) / (2*G)

def MassFromVesc(v_esc, r, a):
    ''' determines the total mass needed for a given escape speed assuming
        assuming a Hernquist profile for the dark matter halo
            M = v_esc**2 * (r+a) / (2*G)
    Inputs:
        v_esc: 'astropy quantity'
            escape speed in km/s (or speed of satellite)
        r: 'astropy quantity'
            distance from Galactic Center (kpc)
        a: 'astropy quantity'
            Hernquist scale length (kpc)
    Outputs:
        M: 'astropy quantity'
            total mass within r (Msun)
    '''
    v_esc_KpcGyr = v_esc.to(u.kpc/u.Gyr) # converting velocity to kpc/Gyr
    M = v_esc_KpcGyr**2 * (r+a) / (2*G)
    return M

In [18]:
v_LeoI = 196 * u.km/u.s # speed of Leo I from Sohn+2013 ApJ 768
a = 30 * u.kpc  # scale radius for Hernquist Halo
r = 260 * u.kpc # galactocentric distance of Leo I

In [23]:
# Compute the mass needed to keep Leo I bound
M_LeoI = MassFromVesc(v_LeoI, r, a)
print(M_LeoI)

1295146976857.1042 solMass


In [26]:
print(f'{M_LeoI:.2e}')

1.30e+12 solMass


In [27]:
M_iso260 / M_LeoI   # isothermal sphere gives larger mass at larger distances

<Quantity 2.57842045>